In [77]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.by import By

In [58]:
df = pd.read_csv("df0_5000.csv")
df.shape

c:\Users\matan\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (17,87) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(4988, 2215)

In [59]:
def listify_geo_markets(df):
    l = []
    for i,val in enumerate(df['geographical markets']):
        if val is not np.nan:
            l.append(val.split(', '))
        else :
            l.append(np.nan)
    return l

In [60]:
def get_location_list(df: pd.DataFrame)-> list : 
    countries = []
    for list in df['geographical markets']:
        if list is not np.nan:
            for country in list:
                if country not in countries:
                    countries.append(country.lower())
    
    return countries

In [61]:
df['geographical markets'] = listify_geo_markets(df)

In [62]:
countries = get_location_list(df)

In [63]:
countries

['australia',
 'canada',
 'france',
 'india',
 'united kingdom',
 'united states',
 'mexico',
 'spain',
 'global',
 'north america',
 'europe',
 'germany',
 'africa',
 'chile',
 'israel',
 'italy',
 'portugal',
 'singapore',
 'south korea',
 'japan',
 'netherlands',
 'south africa',
 'asia',
 'china',
 'central america',
 'south america',
 'southeast asia',
 'brazil',
 'colombia',
 'guatemala',
 'kenya',
 'nigeria',
 'panama',
 'peru',
 'northern europe',
 'oceania',
 'russia',
 'ukraine',
 'united arab emirates',
 'indonesia',
 'philippines',
 'turkey',
 'americas',
 'southern africa',
 'argentina',
 'new zealand',
 'hong kong',
 'cyprus',
 'greece',
 'hungary',
 'moldova',
 'north korea',
 'romania',
 'zambia',
 'west asia',
 'ethiopia',
 "cote d'ivoire",
 'egypt',
 'east asia',
 'sri lanka',
 'malta',
 'ghana',
 'papua new guinea',
 'azerbaijan',
 'belgium',
 'switzerland',
 'cameroon',
 'laos',
 'bulgaria',
 'ecuador',
 'paraguay',
 'uruguay',
 'vietnam',
 'western europe',
 'south

In [64]:
def get_api_data(countries):
    api = "https://restcountries.com/v3.1/"
    api_v = ['name/','region/','subregion/']

    d = {}
    for country in countries:
        time.sleep(0.5)
        for v in api_v:
            r = requests.get(f"{api}{v}{country}")
            if r.status_code == 200:
                d.update({country : r.json()[0]['population']})
                break
    
    return d


In [65]:
# pop_dict

In [78]:
def get_driver():
    from selenium import webdriver
    from selenium.webdriver.firefox.service import Service
    from webdriver_manager.firefox import GeckoDriverManager
    

    options = webdriver.FirefoxOptions()
    options.headless = True
    driver = webdriver.Firefox(options = options, service=Service(GeckoDriverManager().install()))
    return driver

In [67]:
def get_world_pop() -> int :
    """Get world population by scraping www.worldometers.info"""
    
    driver = get_driver()

    url = "https://www.worldometers.info/world-population/"
    driver.get(url)
    population = driver.find_element(By.CLASS_NAME, "maincounter-number").text
    driver.quit()
    pop = population.split(',')
    num = ''.join(pop)
    return int(num)



In [68]:
def get_countries_pop() -> dict:
    """Get countries population by scraping www.worldometers.info"""
    
    
    driver = get_driver()
    url = "https://www.worldometers.info/world-population/population-by-country/"
    driver.get(url)
    d = {}
    table = driver.find_element(By.TAG_NAME, "tbody").find_elements(By.TAG_NAME, "tr")

    for row in table:
        elements = row.find_elements(By.TAG_NAME, "td")
        d.update({elements[1].text.lower(): elements[2].text.lower()})
    
    driver.quit()
    return d
    

In [69]:
country_pop = get_countries_pop()



====== WebDriver manager ======
Current firefox version is 100.0
Get LATEST geckodriver version for 100.0 firefox
Getting latest mozilla release info for v0.31.0
Trying to download new driver from https://github.com/mozilla/geckodriver/releases/download/v0.31.0/geckodriver-v0.31.0-win64.zip
Driver has been saved in cache [C:\Users\matan\.wdm\drivers\geckodriver\win64\v0.31.0]


In [70]:
country_pop.keys()

dict_keys(['china', 'india', 'united states', 'indonesia', 'pakistan', 'brazil', 'nigeria', 'bangladesh', 'russia', 'mexico', 'japan', 'ethiopia', 'philippines', 'egypt', 'vietnam', 'dr congo', 'turkey', 'iran', 'germany', 'thailand', 'united kingdom', 'france', 'italy', 'tanzania', 'south africa', 'myanmar', 'kenya', 'south korea', 'colombia', 'spain', 'uganda', 'argentina', 'algeria', 'sudan', 'ukraine', 'iraq', 'afghanistan', 'poland', 'canada', 'morocco', 'saudi arabia', 'uzbekistan', 'peru', 'angola', 'malaysia', 'mozambique', 'ghana', 'yemen', 'nepal', 'venezuela', 'madagascar', 'cameroon', "côte d'ivoire", 'north korea', 'australia', 'niger', 'taiwan', 'sri lanka', 'burkina faso', 'mali', 'romania', 'malawi', 'chile', 'kazakhstan', 'zambia', 'guatemala', 'ecuador', 'syria', 'netherlands', 'senegal', 'cambodia', 'chad', 'somalia', 'zimbabwe', 'guinea', 'rwanda', 'benin', 'burundi', 'tunisia', 'bolivia', 'belgium', 'haiti', 'cuba', 'south sudan', 'dominican republic', 'czech repub

In [71]:
"he llo".index(' ')

2

In [74]:
def get_not_in_list_data(l: list):
    d = {}
    driver = get_driver()
    url = "https://www.worldometers.info/world-population/"  #  "western-europe-population/"
    postfix=['ern']
    for country in l:
        try:
            sp_index = country.index(' ')
            newcountry = country[sp_index:] + postfix[0] + country[sp_index:]
            new_url = url+ "-".join(newcountry.split(' '))

            population = driver.get(new_url).find_element(By.CLASS_NAME, "maincounter-number").text
            d.update({newcountry: population})

        except:
            continue
    
    driver.quit()
    return d


In [80]:
# List definistions
not_in_list = []

for country in countries:
    if country not in country_pop.keys():
        not_in_list.append(country)

print(len(country_pop))

country_pop.update(get_not_in_list_data(not_in_list))
print(len(country_pop))



====== WebDriver manager ======


235


Current firefox version is 100.0
Get LATEST geckodriver version for 100.0 firefox
Driver [C:\Users\matan\.wdm\drivers\geckodriver\win64\v0.31.0\geckodriver.exe] found in cache


235


In [75]:
country_pop

{'china': '1,439,323,776',
 'india': '1,380,004,385',
 'united states': '331,002,651',
 'indonesia': '273,523,615',
 'pakistan': '220,892,340',
 'brazil': '212,559,417',
 'nigeria': '206,139,589',
 'bangladesh': '164,689,383',
 'russia': '145,934,462',
 'mexico': '128,932,753',
 'japan': '126,476,461',
 'ethiopia': '114,963,588',
 'philippines': '109,581,078',
 'egypt': '102,334,404',
 'vietnam': '97,338,579',
 'dr congo': '89,561,403',
 'turkey': '84,339,067',
 'iran': '83,992,949',
 'germany': '83,783,942',
 'thailand': '69,799,978',
 'united kingdom': '67,886,011',
 'france': '65,273,511',
 'italy': '60,461,826',
 'tanzania': '59,734,218',
 'south africa': '59,308,690',
 'myanmar': '54,409,800',
 'kenya': '53,771,296',
 'south korea': '51,269,185',
 'colombia': '50,882,891',
 'spain': '46,754,778',
 'uganda': '45,741,007',
 'argentina': '45,195,774',
 'algeria': '43,851,044',
 'sudan': '43,849,260',
 'ukraine': '43,733,762',
 'iraq': '40,222,493',
 'afghanistan': '38,928,346',
 'pol

In [79]:
pop = get_world_pop()
pop



====== WebDriver manager ======
Current firefox version is 100.0
Get LATEST geckodriver version for 100.0 firefox
Driver [C:\Users\matan\.wdm\drivers\geckodriver\win64\v0.31.0\geckodriver.exe] found in cache


7946728559

In [38]:

country_pop_percent = {}
for key,val in country_pop.items():
    num = val
    if type(val) == str:
        popu = val.split(',')
        num = int(''.join(popu))
    if type(num) == int:
        country_pop_percent.update({key : num})

country_pop_percent

{'china': 1439323776,
 'india': 1380004385,
 'united states': 331002651,
 'indonesia': 273523615,
 'pakistan': 220892340,
 'brazil': 212559417,
 'nigeria': 206139589,
 'bangladesh': 164689383,
 'russia': 145934462,
 'mexico': 128932753,
 'japan': 126476461,
 'ethiopia': 114963588,
 'philippines': 109581078,
 'egypt': 102334404,
 'vietnam': 97338579,
 'dr congo': 89561403,
 'turkey': 84339067,
 'iran': 83992949,
 'germany': 83783942,
 'thailand': 69799978,
 'united kingdom': 67886011,
 'france': 65273511,
 'italy': 60461826,
 'tanzania': 59734218,
 'south africa': 59308690,
 'myanmar': 54409800,
 'kenya': 53771296,
 'south korea': 51269185,
 'colombia': 50882891,
 'spain': 46754778,
 'uganda': 45741007,
 'argentina': 45195774,
 'algeria': 43851044,
 'sudan': 43849260,
 'ukraine': 43733762,
 'iraq': 40222493,
 'afghanistan': 38928346,
 'poland': 37846611,
 'canada': 37742154,
 'morocco': 36910560,
 'saudi arabia': 34813871,
 'uzbekistan': 33469203,
 'peru': 32971854,
 'angola': 32866272,

In [194]:
country_pop['global'] = pop
country_pop['global']

7946474873

In [197]:
country_pop_percent

{'china': 1439323776,
 'india': 1380004385,
 'united states': 331002651,
 'indonesia': 273523615,
 'pakistan': 220892340,
 'brazil': 212559417,
 'nigeria': 206139589,
 'bangladesh': 164689383,
 'russia': 145934462,
 'mexico': 128932753,
 'japan': 126476461,
 'ethiopia': 114963588,
 'philippines': 109581078,
 'egypt': 102334404,
 'vietnam': 97338579,
 'dr congo': 89561403,
 'turkey': 84339067,
 'iran': 83992949,
 'germany': 83783942,
 'thailand': 69799978,
 'united kingdom': 67886011,
 'france': 65273511,
 'italy': 60461826,
 'tanzania': 59734218,
 'south africa': 59308690,
 'myanmar': 54409800,
 'kenya': 53771296,
 'south korea': 51269185,
 'colombia': 50882891,
 'spain': 46754778,
 'uganda': 45741007,
 'argentina': 45195774,
 'algeria': 43851044,
 'sudan': 43849260,
 'ukraine': 43733762,
 'iraq': 40222493,
 'afghanistan': 38928346,
 'poland': 37846611,
 'canada': 37742154,
 'morocco': 36910560,
 'saudi arabia': 34813871,
 'uzbekistan': 33469203,
 'peru': 32971854,
 'angola': 32866272,

In [41]:
pop_per = {}
sum = 0
sum_per = 0
for key, val in country_pop_percent.items():
    pop_per.update({key.lower(): (val/pop)})
    sum += val
    sum_per += (val/pop)

# pop_per.update({'unknown': ((1-sum_per)*pop) / val })


In [42]:
pop_per

{'china': 0.18112541679451513,
 'india': 0.1736606270105716,
 'united states': 0.041653583524534535,
 'indonesia': 0.03442038518094868,
 'pakistan': 0.027797232156064756,
 'brazil': 0.026748611840984517,
 'nigeria': 0.025940736613993827,
 'bangladesh': 0.02072461640313134,
 'russia': 0.01836448525007436,
 'mexico': 0.01622497940698874,
 'japan': 0.015915878063922317,
 'ethiopia': 0.014467090824109973,
 'philippines': 0.013789752352108907,
 'egypt': 0.012877826300090451,
 'vietnam': 0.01224914851372596,
 'dr congo': 0.011270463753581834,
 'turkey': 0.010613281679323511,
 'iran': 0.010569725970694626,
 'germany': 0.010543424397261872,
 'thailand': 0.008783673498837545,
 'united kingdom': 0.008542818677714972,
 'france': 0.008214060020861053,
 'italy': 0.007608554528878603,
 'tanzania': 0.007516991876707822,
 'south africa': 0.007463443163316919,
 'myanmar': 0.006846963738828845,
 'kenya': 0.006766613990528038,
 'south korea': 0.006451746755443095,
 'colombia': 0.006403135273494491,
 'spa

In [47]:
for i, list in enumerate(df['geographical markets']):
    sum = 0
    if list is not np.nan :
        for country in list :
            if country in pop_per.keys() :
                sum += pop_per[country]
        df.loc[i,'geo_market_per'] = sum if sum < 1 else 1


df['geo_market_per']



0       2.400295e-01
1                NaN
2       6.851171e-02
3       1.000000e+00
4       1.000000e+00
            ...     
4983             NaN
4984    1.000000e+00
4985             NaN
4986    7.637268e-07
4987             NaN
Name: geo_market_per, Length: 4988, dtype: float64

In [52]:
df.loc[4986,['geographical markets','geo_market_per']]

geographical markets    [north america]
geo_market_per                 0.000001
Name: 4986, dtype: object